In [47]:
!pip install tweepy
!pip3 install -U scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [17]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime
from pymongo import MongoClient
import json

In [6]:
consumer_key='CepRIvrohFCwvM9Wgt1yrRd7V'

In [7]:
consumer_secret='hlOfF8XoYuI7kVMP2xKCXjTPW5jYQ6eTMMZosyLOHjOTQGBpOM'

In [8]:
access_token='1104878384956481543-F8N0LQfPphbs4k1ZATRYBFibWfICZy'

In [9]:
access_token_secret='On5sB0nNmtRjypZ1193Vjm5wownZQZbYNmdeQSilmA0mA'

In [10]:
auth= OAuthHandler(consumer_key, consumer_secret)

In [12]:
auth.set_access_token(access_token, access_token_secret)

In [14]:
# criando uma classe para capturar os stream de dados do tweeter e
# armazenar no MongoDB

class MyListener(StreamListener):
    def on_data(self, dados):
        tweet= json.loads(dados)
        created_at= tweet['created_at']
        id_str= tweet['id_str']
        text= tweet['text']
        obj= {'created_at':created_at,'id_str':id_str,'text':text,}
        tweetind= col.insert_one(obj).inserted_id
        print(obj)
        return True

In [15]:
myListener= MyListener()

In [16]:
mystream= Stream(auth, listener= myListener)

## conectando com o Mongo

In [18]:
client_con= MongoClient('localhost', 6660)

In [19]:
db= client_con.twitterdb

In [20]:
col= db.tweets

In [21]:
keywords= ['Big Data', 'Python', 'Data Mining', 'Data Science']

In [22]:
mystream.filter(track=keywords)

{'created_at': 'Tue Feb 16 00:49:51 +0000 2021', 'id_str': '1361477858309251079', 'text': 'RT @GovRonDeSantis: Today we take another important step to hold Big Tech accountable and shift the balance of power back to consumers with…', '_id': ObjectId('602b16b480be68393aba4296')}
{'created_at': 'Tue Feb 16 00:49:57 +0000 2021', 'id_str': '1361477885635166208', 'text': 'RT @quay_dr: @quay_dr @DrLiMengYAN1 @ZENODO_ORG Dr. Yan. I appreciate your support and commend you for your honesty and courage.', '_id': ObjectId('602b16ba80be68393aba4297')}
{'created_at': 'Tue Feb 16 00:49:58 +0000 2021', 'id_str': '1361477890597134338', 'text': "RT @Al_Grigor: A list of coding problems you're likely to encounter on a data science interview\n\nhttps://t.co/uQ3oBEwfZE", '_id': ObjectId('602b16bb80be68393aba4298')}
{'created_at': 'Tue Feb 16 00:49:59 +0000 2021', 'id_str': '1361477894464299010', 'text': '@IOHK_Charles dev position (python 10/10, golang 5/10, Rust Haskel experience but want to learn)  no m

{'created_at': 'Tue Feb 16 00:50:22 +0000 2021', 'id_str': '1361477991428145153', 'text': 'RT @RepAndyBiggsAZ: I was one of the first public officials last year to look at the science &amp; data &amp; call for schools to give working pare…', '_id': ObjectId('602b16d480be68393aba42b4')}
{'created_at': 'Tue Feb 16 00:50:23 +0000 2021', 'id_str': '1361477996092166150', 'text': 'RT @flame_proxies: BIG GIVEAWAY 🎉\n\n@RhinoProxies - 1GB data\n@PluggedRobotics - 1 Free month\n@ProxiesPlug - 1GB data\n@flame_proxies - 1GB da…', '_id': ObjectId('602b16d580be68393aba42b5')}
{'created_at': 'Tue Feb 16 00:50:25 +0000 2021', 'id_str': '1361478003419717635', 'text': 'RT @ChrisSprowls: Great to stand with @GovRonDeSantis, @LtGovNunez and @FionaForFlorida today on behalf of Florida consumers as we unveiled…', '_id': ObjectId('602b16d680be68393aba42b6')}
{'created_at': 'Tue Feb 16 00:50:26 +0000 2021', 'id_str': '1361478008503214081', 'text': 'RT @glitchbotio: "What kills the creative force is not age 

{'created_at': 'Tue Feb 16 00:50:55 +0000 2021', 'id_str': '1361478126698659841', 'text': 'RT @SteffnWasson: @RKJ65 If his video geotag data isn’t being stripped (Parler’s big FUBAR), the FBI already knows his whereabouts.', '_id': ObjectId('602b16f480be68393aba42d3')}
{'created_at': 'Tue Feb 16 00:50:56 +0000 2021', 'id_str': '1361478131324968960', 'text': 'RT @serundeputy: MA 2 Wk positivit, cases down, but average deaths over 2 Wk avervages are even.\n\nhttps://t.co/q9tTSWLV8U\n\n#COVID19 #python…', '_id': ObjectId('602b16f580be68393aba42d4')}
{'created_at': 'Tue Feb 16 00:50:58 +0000 2021', 'id_str': '1361478139197747206', 'text': 'RT @TDataScience: How to Make Stunning Radar Charts with Python\u200a—\u200aImplemented in Matplotlib and Plotly by Dario Radečić\nhttps://t.co/rzdCVH…', '_id': ObjectId('602b16f780be68393aba42d5')}
{'created_at': 'Tue Feb 16 00:50:59 +0000 2021', 'id_str': '1361478145514291200', 'text': 'RT @HarbRimah: Outsmarting ML Biases https://t.co/UaJuEhcZvq #Mach

{'created_at': 'Tue Feb 16 00:51:23 +0000 2021', 'id_str': '1361478244109676544', 'text': '@missdearyme @IanDunt Thats brilliant! So proud 😀😊\n\nWe do the same. Its like Monty Python, you know immediately who… https://t.co/QU9jG2Vc93', '_id': ObjectId('602b171080be68393aba42f2')}
{'created_at': 'Tue Feb 16 00:51:24 +0000 2021', 'id_str': '1361478249516322829', 'text': 'RT @freecodecoupon: #Statistics for Data Science and Business Analysis #Udemy\n\nPreview This Course GET COUPON CODE =&gt; https://t.co/bayNZgGl…', '_id': ObjectId('602b171180be68393aba42f3')}
{'created_at': 'Tue Feb 16 00:51:24 +0000 2021', 'id_str': '1361478249763774465', 'text': 'RT @freecodecoupon: #Statistics for Data Science and Business Analysis #Udemy\n\nPreview This Course GET COUPON CODE =&gt; https://t.co/bayNZgGl…', '_id': ObjectId('602b171180be68393aba42f4')}
{'created_at': 'Tue Feb 16 00:51:27 +0000 2021', 'id_str': '1361478263542022146', 'text': 'RT @progbooks1: (Introducing Delphi ORM) Check it out 👉 - ht

{'created_at': 'Tue Feb 16 00:51:44 +0000 2021', 'id_str': '1361478334388056064', 'text': 'RT @Eli_Krumova: #Data Pre-processing for #MachineLearning \nhttps://t.co/pkPm6Fqdiu\n\n#Analytics #BigData #ML #AI #ArtificialIntelligence #P…', '_id': ObjectId('602b172680be68393aba4310')}
{'created_at': 'Tue Feb 16 00:51:44 +0000 2021', 'id_str': '1361478334450966532', 'text': "RT @KavitaG20503846: Today's Quiz on Machine Learning!!\n#Statistics #machinelearning #100daysofmlcode #100DaysOfCode  #serverless  #ai http…", '_id': ObjectId('602b172680be68393aba4311')}
{'created_at': 'Tue Feb 16 00:51:46 +0000 2021', 'id_str': '1361478341816184834', 'text': 'RT @Fibci2: Why can’t we have a leader like him in Massachusetts. @MassGovernor has never cared to address these issues. He can’t even get…', '_id': ObjectId('602b172780be68393aba4312')}
{'created_at': 'Tue Feb 16 00:51:49 +0000 2021', 'id_str': '1361478354713591809', 'text': 'RT @freecodecoupon: #Statistics for Data Science and Business Analysi

KeyboardInterrupt: 

In [24]:
mystream.disconnect()

In [25]:
col.find_one()

{'_id': ObjectId('602b16b480be68393aba4296'),
 'created_at': 'Tue Feb 16 00:49:51 +0000 2021',
 'id_str': '1361477858309251079',
 'text': 'RT @GovRonDeSantis: Today we take another important step to hold Big Tech accountable and shift the balance of power back to consumers with…'}

## Usando o pandas e scikit-learn para analisar os dados coletados

In [41]:
#criando o dataset com os dados retornados do mongoDB
dataset= [{'created_at':item ['created_at'], 'text':item['text'],} for item in col.find()]

In [42]:
import pandas as pd

In [43]:
df= pd.DataFrame(dataset)

In [44]:
df

,created_at,text
0,Tue Feb 16 00:49:51 +0000 2021,RT @GovRonDeSantis: Today we take another impo...
1,Tue Feb 16 00:49:57 +0000 2021,RT @quay_dr: @quay_dr @DrLiMengYAN1 @ZENODO_OR...
2,Tue Feb 16 00:49:58 +0000 2021,RT @Al_Grigor: A list of coding problems you'r...
3,Tue Feb 16 00:49:59 +0000 2021,"@IOHK_Charles dev position (python 10/10, gola..."
4,Tue Feb 16 00:50:00 +0000 2021,RT @Stella72314914: @Ieafmustard @lingwens Nee...
...,...,...
142,Tue Feb 16 00:52:08 +0000 2021,RT @MiriamAsensi: HCL OneDB the database for c...
143,Tue Feb 16 00:52:09 +0000 2021,RT @AssignHelp001: We offer professional help;...
144,Tue Feb 16 00:52:09 +0000 2021,RT @AssignHelp001: We offer professional help;...
145,Tue Feb 16 00:52:09 +0000 2021,RT @AssignHelp001: We offer professional help;...


In [48]:
from sklearn.feature_extraction.text import CountVectorizer

In [51]:
cv= CountVectorizer()

In [52]:
count_matrix= cv.fit_transform(df.text)

In [53]:
word_count= pd.DataFrame(cv.get_feature_names(), columns=['word'])
word_count['count']= count_matrix.sum(axis=0).tolist()[0]
word_count= word_count.sort_values('count', ascending=False).reset_index(drop=True)
word_count[:50]

,word,count
0,rt,126
1,https,64
2,co,64
3,to,47
4,the,45
5,data,44
6,and,40
7,for,38
8,python,38
9,of,36


# Fim